In [1]:
import numpy as np
import cv2
import os

## Load Images

In [2]:
img_root = './images'
align_img_root = './align_images'
img_names = ['1.jpg', '2.jpg']
# img_names = ['test_1.png', 'test_2.png']

In [3]:
src_imgs = []
for img_name in img_names:
    img = cv2.imread(os.path.join(img_root, img_name))
    src_imgs.append(img)

## Median Threshold Bitmap Algorithm

In [4]:
def getBitmap(img):
    img = img.astype(np.float32) / 255
    gray_img = (54*img[:, :, 2] + 183*img[:, :, 1] + 19*img[:, :, 0]) / 256
    median = np.median(gray_img)
    binary_img = (gray_img > median) * 1.0
    mask_img = np.invert(cv2.inRange(gray_img, median-0.0156, median+0.0156).astype(bool)) * 1.0
    return binary_img, mask_img

In [5]:
def bitmapShift(img, xs, ys):
    h, w = img.shape[:2]
    M = np.float32([[1, 0, xs], [0, 1, ys]])
    shift_img = cv2.warpAffine(img, M, (w, h))
    return shift_img

In [6]:
def getShift(img1, img2, shift_bits):
    
    if shift_bits > 0:
        h, w = img1.shape[:2]
        sm_h, sm_w = int(h/2), int(w/2)
        sm_img1 = cv2.resize(img1, (sm_w, sm_h))
        sm_img2 = cv2.resize(img2, (sm_w, sm_h))
        cur_shift = getShift(sm_img1, sm_img2, shift_bits-1)
        cur_shift = 2 * cur_shift
    else:
        cur_shift = np.zeros([2], dtype=int)
    
    min_err = np.inf
    best_shift = cur_shift
    binary_img1, mask1 = getBitmap(img1)
    binary_img2, mask2 = getBitmap(img2)
    row, col = np.mgrid[-1:2, -1:2]
    for i, j in zip(row.reshape(-1,), col.reshape(-1,)):
        ys, xs = cur_shift[0]+i, cur_shift[1]+j
        shift_img2 = bitmapShift(binary_img2.copy(), xs, ys)
        shift_mask2 = bitmapShift(mask2.copy(), xs, ys)
        diff_b = cv2.bitwise_xor(binary_img1, shift_img2)
        diff_b = cv2.bitwise_and(diff_b, mask1)
        diff_b = cv2.bitwise_and(diff_b, shift_mask2)
        err = np.sum(diff_b)
        if err < min_err:
            min_err = err
            best_shift[0] = ys
            best_shift[1] = xs
    return best_shift

## Start Aiigning

In [7]:
align_imgs = [src_imgs[0]]
for src_img in src_imgs[1:]:
    ys, xs = getShift(src_imgs[0], src_img, 6)
    print(xs, ys)
    align_img = bitmapShift(src_img, xs, ys)
    align_imgs.append(align_img)

-20 -6


In [8]:
cv2.imshow('img', align_imgs[0])
cv2.imshow('img2', align_imgs[1])
cv2.waitKey(0)

-1

## Blend Two Images to Check Alignment Effect

In [ ]:
blend_img = (align_imgs[0].astype(np.float32))/255 * 0.5 + (align_imgs[1].astype(np.float32))/255 * 0.5
cv2.imshow('img', blend_img)
cv2.waitKey(0)

## Save Aligned Result

In [ ]:
for align_img, img_name in zip(align_imgs, img_names):
    cv2.imwrite(os.path.join(align_img_root, img_name), align_img)

## OpenCV Official Implementation

In [ ]:
alignMTB = cv2.createAlignMTB()
alignMTB.process(src_imgs, src_imgs)

In [ ]:
cv2.imshow('img', src_imgs[0])
cv2.imshow('img2', src_imgs[1])
cv2.waitKey(0)